<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg"  width=300, align="right">
<br>
<br>
<br>
<br>
<br>

# **Expert Systems, Rule‑Based Programming `PyKnown`**
---


In [1]:
#@title **Identificação do Grupo**

#@markdown Integrantes do Grupo, nome completo em orgem alfabética (*informe \<RA\>,\<nome\>*)
Aluno1 = '10403025, Gabriel Gonzaga Chung' #@param {type:"string"}
Aluno2 = '10403462, Igor Benites Moura' #@param {type:"string"}
Aluno3 = '10401873, Rodrigo Machado de Assis de Oliveira' #@param {type:"string"}
Aluno4 = 'None' #@param {type:"string"}
Aluno5 = 'None' #@param {type:"string"}



# **Resumo**

*Apresente um parágrafo resumindo o que é feito neste trabalho*.

Neste trabalho é montada uma representação simples de uma classificação de alimentos, utilizando um conjunto de regras presentes no ``pyknow``. A partir disso, pode ser inferido determinadas informações destes alimentos, como as características e o tipo do alimento.

# **Implementação**

*Aqui o código da sua solução. Você deve explicar em linhas gerais o código. Para isso você pode quebrar o código em algumas partes e incluir células de texto explicativos. Não empregue para isso comentários no corpo do código, mas crie células de texto do notebook.*

> ***ATENÇÃO: O código deve ser 100% executável no Colab e, portanto, não devem ser empregados arquivos locais (da sua máquina) e, se necessário, devem ser incluídos o install de novas bibliotecas empregadas.***



Primeiro desinstalamos um pacote incompatível (**`yfinance`**) com o que usaremos, neste caso o **`pyknow`**.

In [ ]:
!pip uninstall yfinance

Found existing installation: yfinance 0.2.55
Uninstalling yfinance-0.2.55:
  Would remove:
    /usr/local/bin/sample
    /usr/local/lib/python3.11/dist-packages/yfinance-0.2.55.dist-info/*
    /usr/local/lib/python3.11/dist-packages/yfinance/*
Proceed (Y/n)? y
  Successfully uninstalled yfinance-0.2.55


Depois são importados os pacotes utilizados, neste caso **`sys`** e **`pyknow`** e **`collections`**.
\
***OBS: Como não tem o pacote do `pyknow` instalado nativamente é preciso instalá-lo.`***

In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/
import collections.abc
collections.Mapping = collections.abc.Mapping

from pyknow import *

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-rhrl7xvk
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-rhrl7xvk
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done


Neste código é definido um conjunto de regras buscando representar uma classificação simplificada de **alimentos**.
A partir de certas **características *(regras)*** :
-  É inferido o **tipo do alimento**. Podendo ser **carne, vegetal, laticínio** ou **grão**. Ex: se temos um alimento com **fibra alta** e **amido**, pode ser inferido o tipo **grão**.

- Podem ser inferidas outras **características**. Ex: se temos um alimento que é uma **fritura**, ele também contém **gordura alta**.

- É informado se o alimento é **saudável** e algumas características relevantes a alérgicos. Neste modelo, são apenas considerados o **glutén** e a **lactose**.

In [ ]:
class Alimentos(KnowledgeEngine):
    @Rule(AND(NOT(Fact(tipo='vegetal')),
              NOT(Fact(tipo='laticinio')),
              NOT(Fact(tipo='grao')),
              NOT(Fact(tipo='fibra_alta')),
              NOT(Fact('amido')),
              OR(Fact('proteina_alta'), Fact('gordura_alta'))))
    def assume_carne(self):
        self.declare(Fact(tipo='carne'))

    @Rule(AND(
        Fact('fibra_alta'),
        NOT(Fact('amido')),
        NOT(Fact('gluten')),
        NOT(OR(Fact(tipo='grão'), Fact(tipo='laticínio'), Fact(tipo='carne')))
    ))
    def assume_vegetal(self):
        self.declare(Fact(tipo='vegetal'))

    @Rule(OR(AND(
              Fact('amido'),
              NOT(OR(
                  Fact(tipo='vegetal'),
                  Fact(tipo='laticínio'),
                  Fact(tipo='carne')))),
             Fact('gluten')))
    def assume_grao(self):
        self.declare(Fact(tipo='grao'))

    @Rule(AND(
        Fact('lactose'),
        NOT(OR(Fact(tipo='vegetal'), Fact(tipo='grao'), Fact(tipo='carne')))
    ))
    def assume_laticinio(self):
        self.declare(Fact(tipo='laticinio'))

    @Rule(AND(Fact(tipo='vegetal'), Fact('proteina_alta')))
    def proteina_vegana(self):
        self.declare(Fact('proteina_vegetal'))

    @Rule(AND(Fact(tipo='grao'), Fact('proteina_alta')))
    def grao_fibra(self):
        self.declare(Fact('grao_proteico'))

    @Rule(OR(AND(Fact('natural'), Fact('fibra_alta')), Fact('organico')))
    def saudavel(self):
        self.declare(Fact('muito_saudavel'))

    @Rule(OR(Fact('processado'), Fact('industrializado'), Fact('acucar_alto')))
    def nao_saudavel(self):
        self.declare(Fact('nao_saudavel'))

    @Rule(Fact('fritura'))
    def fritura_gordura_alta(self):
        self.declare(Fact('gordura_alta'))

    @Rule(AND(Fact('fritura'), Fact('gordura_alta')))
    def fritura_nao_saudavel(self):
        self.declare(Fact('nao_saudavel'))

    @Rule(AND(Fact('fast_food'), OR(Fact('processado'), Fact('gordura_alta'))))
    def fast_food_ruim(self):
        self.declare(Fact('nao_saudavel'))

    @Rule(Fact('organico'))
    def organico_natural(self):
        self.declare(Fact('natural'))

    @Rule(Fact('natural'))
    def natural_muito_saudavel(self):
        self.declare(Fact('muito_saudavel'))

    @Rule(AND(Fact(tipo='grao'), Fact('gluten')))
    def contem_gluten(self):
        self.declare(Fact('restricao_gluten'))

    @Rule(AND(Fact(tipo='laticinio'), Fact('lactose')))
    def contem_lactose(self):
        self.declare(Fact('restricao_lactose'))

    @Rule(NOT(Fact('nao_saudavel')))
    def saudavel(self):
        self.declare(Fact('saudavel'))

    @Rule(Fact('saudavel'))
    def imprimir_saudavel(self):
        print("Este alimento é saudável.")

    @Rule(Fact('nao_saudavel'))
    def informar_nao_saudavel(self):
        print("Este alimento não é saudável.")

    @Rule(Fact('restricao_lactose'))
    def informar_restricao_lactose(self):
        print("Este alimento contém lactose.")

    @Rule(Fact('restricao_gluten'))
    def informar_restricao_gluten(self):
        print("Este alimento contém gluten.")

    @Rule(Fact(tipo=MATCH.a))
    def informar_resultado(self,a):
          print('Alimento é do tipo: {}'.format(a))

    def declarar_fatos(self, lista_fatos):
        for fato in lista_fatos:
            self.declare(fato)

# **Resultados (Testes)**

*Um ou mais testes da solução devem ser apresentados ao final. Comente em células de texto explicativas os testes e os seus resultados.*

**Testes realizados:**

Testando para ver se o alimento se trata de **carne**.

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('proteina_alta'),
    Fact('gordura_alta'),
])
exemplo.run()
exemplo.facts

Alimento é do tipo: carne
Este alimento é saudável.


FactList([(0, InitialFact()),
          (1, Fact('proteina_alta')),
          (2, Fact('gordura_alta')),
          (3, Fact(tipo='carne')),
          (4, Fact('saudavel'))])

Verificando se o alimento se trata de **grão**.

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('gluten'),
    Fact('fibra_alta')
])
exemplo.run()
exemplo.facts

Este alimento contém gluten.
Alimento é do tipo: grao
Este alimento é saudável.


FactList([(0, InitialFact()),
          (1, Fact('gluten')),
          (2, Fact('fibra_alta')),
          (3, Fact(tipo='grao')),
          (4, Fact('restricao_gluten')),
          (5, Fact('saudavel'))])

Verificando se o alimento se trata de **vegetal**.

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('fibra_alta')
])
exemplo.run()
exemplo.facts

Alimento é do tipo: vegetal
Este alimento é saudável.


FactList([(0, InitialFact()),
          (1, Fact('fibra_alta')),
          (2, Fact(tipo='vegetal')),
          (3, Fact('saudavel'))])

Verificando se o alimento se trata de **proteína vegetariana**.

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('proteina_alta'),
    Fact('fibra_alta'),
])
exemplo.run()
exemplo.facts

Alimento é do tipo: vegetal
Este alimento é saudável.


FactList([(0, InitialFact()),
          (1, Fact('proteina_alta')),
          (2, Fact('fibra_alta')),
          (3, Fact(tipo='vegetal')),
          (4, Fact('proteina_vegetal')),
          (5, Fact('saudavel'))])

Verificando se o alimento seria um **laticínio saudável**.

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('proteina_alta'),
    Fact('lactose'),
    Fact('natural')
])
exemplo.run()
exemplo.facts

Este alimento contém lactose.
Alimento é do tipo: laticinio
Este alimento é saudável.


FactList([(0, InitialFact()),
          (1, Fact('proteina_alta')),
          (2, Fact('lactose')),
          (3, Fact('natural')),
          (4, Fact('muito_saudavel')),
          (5, Fact(tipo='laticinio')),
          (6, Fact('restricao_lactose')),
          (7, Fact('saudavel'))])

Verificando se o alimento que seria uma **fritura de carne é saudável**.
\
***Obs: Neste caso não é para ser.***

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('proteina_alta'),
    Fact('fritura')
])
exemplo.run()
exemplo.facts

Este alimento não é saudável.
Alimento é do tipo: carne


FactList([(0, InitialFact()),
          (1, Fact('proteina_alta')),
          (2, Fact('fritura')),
          (3, Fact('gordura_alta')),
          (4, Fact('nao_saudavel')),
          (5, Fact(tipo='carne'))])

Verificando se o alimento que seria um alimento **processado é saudável**.
\
***Obs: Neste caso também não é para ser.***

In [ ]:
exemplo = Alimentos()
exemplo.reset()
exemplo.declarar_fatos([
    Fact('amido'),
    Fact('processado')
])
exemplo.run()
exemplo.facts

# **Referências**

Este é um item obrigatório. Inclua aqui o as referências, fontes, ou bibliografia e sites/bibliotecas empregadas em formato ABNT.

**PyKnow: Expert Systems for Python**. Disponível em: <https://github.com/buguroo/pyknow>. Acesso em: 24 mar. 2025.



---

In [2]:
#@title **Avaliação**
Resumo = 10 #@param {type:"slider", min:0, max:10, step:1}

Implementacao = 10 #@param {type:"slider", min:0, max:10, step:1}

Resultados = 10 #@param {type:"slider", min:0, max:10, step:1}

Referencias = 10 #@param {type:"slider", min:0, max:10, step:1}

Geral = 10 #@param {type:"slider", min:0, max:10, step:1}








In [3]:
#@title **Nota Final**
nota = Resumo + Implementacao + Resultados + Referencias + Geral

nota = nota / 5

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_tia = []
lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_tia.append(lista[0]); lista_nome.append(lista[1].upper())")

alunos['tia'] = lista_tia
alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 10.0



tia                                   nome  nota
0  10403025                  GABRIEL GONZAGA CHUNG  10.0
1  10403462                     IGOR BENITES MOURA  10.0
2  10401873   RODRIGO MACHADO DE ASSIS DE OLIVEIRA  10.0